Notebook que permite generar los datos desde un notebook, por ejemplo, en Colab

In [1]:
from pathlib import Path
import os
import IPython

def detectar_entorno_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

def configurar_rutas(base_local=None, base_colab="/content/drive/MyDrive/ia_thermal_colab", verbose=True):
    IN_COLAB = detectar_entorno_colab()

    if IN_COLAB:
        from google.colab import drive
        drive.mount('/content/drive')
        base_path = Path(base_colab)
    else:
        # ⚠️ Si no se pasa base_local, usar el directorio del notebook
        if base_local is None:
            base_path = Path.cwd()
        else:
            base_path = Path(base_local).expanduser().resolve()

    datasets_path = base_path / "datasets"
    models_path = base_path / "models"
    datasets_path.mkdir(parents=True, exist_ok=True)
    models_path.mkdir(parents=True, exist_ok=True)

    if verbose:
        print("✅ Entorno detectado:", "Colab" if IN_COLAB else "Local")
        print("📁 Ruta base:", base_path)
        print("📁 Ruta datasets:", datasets_path)
        print("📁 Ruta modelos:", models_path)

    return IN_COLAB, base_path, datasets_path, models_path

# 🟢 Llamada principal
IN_COLAB, BASE_PATH, DATASETS_PATH, MODELS_PATH = configurar_rutas()


✅ Entorno detectado: Local
📁 Ruta base: c:\Users\ismael.gallo\Desktop\ia_thermal\ismaelgallo
📁 Ruta datasets: c:\Users\ismael.gallo\Desktop\ia_thermal\ismaelgallo\datasets
📁 Ruta modelos: c:\Users\ismael.gallo\Desktop\ia_thermal\ismaelgallo\models


In [2]:
# 🔄 Parámetros del repositorio
GIT_REPO_URL = "https://github.com/ismaelgallolopez/ia_thermal.git"  # 👈 Cambia si usas otro repo
REPO_NAME = GIT_REPO_URL.split("/")[-1].replace(".git", "")
CLONE_PATH = BASE_PATH / REPO_NAME  # Usamos Path (de pathlib)

def clonar_repo_si_no_existe(repo_url: str, clone_path: Path):
    if not clone_path.exists():
        print(f"📥 Clonando repositorio en {clone_path}...")
        os.system(f"git clone {repo_url} {clone_path}")
    else:
        print(f"📂 Repositorio ya clonado en: {clone_path}")

def instalar_requirements(clone_path: Path):
    req_path = clone_path / "requirements.txt"
    if req_path.exists():
        print("📦 Instalando dependencias desde requirements.txt...")
        os.system(f"pip install -r {req_path}")
    else:
        print("⚠️ No se encontró requirements.txt en el repositorio.")

def reiniciar_entorno_colab():
    print("🔄 Reiniciando entorno para aplicar los cambios...")
    display(IPython.display.Javascript('''google.colab.restartRuntime()'''))

# 🧪 Ejecutar solo en Colab
if IN_COLAB:
    clonar_repo_si_no_existe(GIT_REPO_URL, CLONE_PATH)
    instalar_requirements(CLONE_PATH)
    reiniciar_entorno_colab()

In [ ]:
# Librerías estándar
import os, sys, time, json, platform
from datetime import datetime
from typing import Sequence, Union, Optional


# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, Markdown

# NumPy y ciencia de datos
import numpy as np

# PyTorch y entrenamiento
import torch
from torch import nn, amp
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch._dynamo
from contextlib import nullcontext

# Optimización
import optuna

# Añadir rutas del proyecto (de forma portable)
sys.path.append(str(BASE_PATH))
sys.path.append(str(BASE_PATH / "ia_thermal"))
sys.path.append(str(BASE_PATH / "ia_thermal" / "ismaelgallo"))
sys.path.append(str(BASE_PATH.parent))

# Módulos propios del proyecto
from architectures.convlstm import *
from Dataset_Class_convlstm import *
from plot_functions import *
from Physics_Loss import *
from utils import *
from scripts.PCB_solver_tr import *

In [ ]:
# %%
# Montar Google Drive y configurar rutas en Colab
def mount_drive():
    try:
        from google.colab import drive
        drive.mount('/content/drive')
    except ImportError:
        print("No se detecta entorno Colab. Asegúrate de montar tu directorio manualmente si usas otro entorno.")

mount_drive()

import numpy as np
import os
import sys
import time
import torch
from pathlib import Path

# Ajusta la ruta al directorio de scripts si es necesario
scripts_dir = Path('scripts').resolve()
if scripts_dir.exists():
    sys.path.append(str(scripts_dir))

# Directorio en MyDrive para guardar datasets
drive_root = Path('/content/drive/MyDrive')
datasets_dir = drive_root / 'datasets'
datasets_dir.mkdir(parents=True, exist_ok=True)
print(f"📂 Guardando datasets en: {datasets_dir}")

# %%
# # Importar módulos del proyecto
# from PCB_solver_tr import PCB_case_2
# from Dataset_Class_convlstm import PCBDataset_convlstm

# %%
# Parámetros de generación
torch.manual_seed(0)
numpy_seed = np.random.seed(0)
solver = 'transient'  # 'steady' o 'transient'
n_train = 2500
n_validation = 500
n_test = 50
n_data = n_train + n_test + n_validation

time_sim = 500
dt = 1
T_init = 298.0
nodes_side = 13

# Índices de partición
eval_slice = slice(n_test + n_train, n_data)
train_slice = slice(n_test, n_test + n_train)
test_slice  = slice(0, n_test)

# %%
def generate_unique_cases(n_samples):
    seen = set()
    Q_list, T_int_list, T_env_list = [], [], []
    while len(Q_list) < n_samples:
        Q = tuple(np.random.uniform(0.5, 1.5, 4).round(6))
        T_int = tuple(np.random.uniform(270, 320, 4).round(2))
        T_env = round(float(np.random.uniform(270, 320)), 2)
        key = Q + T_int + (T_env,)
        if key not in seen:
            seen.add(key)
            Q_list.append(Q)
            T_int_list.append(T_int)
            T_env_list.append(T_env)
    return np.array(Q_list), np.array(T_int_list), np.array(T_env_list)

Q_rand, T_int_rand, T_env_rand = generate_unique_cases(n_data)

# %%
# Generación de secuencias de entrada y salida
time_start = time.time()
input_seq, output_seq = [], []
for i in range(n_data):
    if i % 100 == 0:
        print(f"Generando caso {i}/{n_data}, tiempo: {time.time() - time_start:.1f}s")
    T_sim, *_ = PCB_case_2(
        solver=solver, display=False, time=time_sim, dt=dt, T_init=T_init,
        Q_heaters=Q_rand[i], T_interfaces=T_int_rand[i], Tenv=T_env_rand[i]
    )
    T_sim = T_sim.reshape(-1, nodes_side, nodes_side).astype(np.float32)
    output_seq.append(T_sim)

    seq_len = T_sim.shape[0]
    case_inputs = []
    for t in range(seq_len):
        T_map = np.zeros((nodes_side, nodes_side), dtype=np.float32)
        Q_map = np.zeros((nodes_side, nodes_side), dtype=np.float32)
        T_env_map = np.full((nodes_side, nodes_side), T_env_rand[i], dtype=np.float32)
        T_map[0,0], T_map[0,-1], T_map[-1,-1], T_map[-1,0] = T_int_rand[i]
        Q_map[6,3], Q_map[3,6], Q_map[9,3], Q_map[9,9] = Q_rand[i]
        T_prev = np.full((nodes_side, nodes_side), T_init, dtype=np.float32) if t == 0 else output_seq[-1][t-1]
        case_inputs.append(np.stack([T_map, Q_map, T_env_map, T_prev], axis=0))
    input_seq.append(np.stack(case_inputs, axis=0))
print(f"Tiempo generación datos: {time.time() - time_start:.1f}s")

# %%
# Convertir a tensores y crear datasets
a = np.array(input_seq, dtype=np.float32)
b = np.array(output_seq, dtype=np.float32)
stats = {
    'T_int_mean': T_int_rand.mean(), 'T_int_std': T_int_rand.std(),
    'Q_mean': Q_rand.mean(),         'Q_std': Q_rand.std(),
    'T_env_mean': T_env_rand.mean(),  'T_env_std': T_env_rand.std(),
    'out_mean': b.mean(),            'out_std': b.std()
}
Tensor_in = torch.tensor(a)
Tensor_out = torch.tensor(b)

def make_dataset(idx):
    return PCBDataset_convlstm(
        T_interfaces=Tensor_in[idx,:,0], Q_heaters=Tensor_in[idx,:,1],
        T_env=Tensor_in[idx,:,2],       T_outputs=Tensor_out[idx],
        T_interfaces_mean=stats['T_int_mean'], T_interfaces_std=stats['T_int_std'],
        Q_heaters_mean=stats['Q_mean'],         Q_heaters_std=stats['Q_std'],
        T_env_mean=stats['T_env_mean'],         T_env_std=stats['T_env_std'],
        T_outputs_mean=stats['out_mean'],       T_outputs_std=stats['out_std'],
        return_bc=True
    )

# Crear splits
dataset_train = make_dataset(train_slice)
dataset_val   = make_dataset(eval_slice)
dataset_test  = make_dataset(test_slice)

# %%
# Guardar datasets en MyDrive
time_save = time.time()
for split, ds in [('train', dataset_train), ('val', dataset_val), ('test', dataset_test), ('full', make_dataset(slice(None)))]:
    file = datasets_dir / f"PCB_convlstm_phy_6ch_{solver}_dataset_{split}.pth"
    torch.save(ds, file)
    print(f"✅ Guardado '{file.name}' en '{file}' ({len(ds)} muestras)")
print(f"Tiempo total guardado: {time.time() - time_save:.1f}s")

# %%
print("¡Dataset listo para usar en el notebook desde MyDrive!")
